In [ ]:
import numpy as np
import os
import plotly.graph_objects as go
from tqdm import tqdm
import pandas as pd

import chimera_fgo.util.general as general
from chimera_fgo.util.io import read_lidar_bin, read_gt
from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results, load_sv_positions
from chimera_fgo.util.geometry import euler_to_R

import symforce
try:
    symforce.set_epsilon_to_symbol()
except symforce.AlreadyUsedEpsilon:
    print("Already set symforce epsilon")
    pass 

import symforce.symbolic as sf

%load_ext autoreload
%autoreload 2

np.set_printoptions(suppress=True, precision=3)

### Load data

In [ ]:
kitti_seq = '0034'
start_idx = 0

gtpath = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

data_path = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'icp')
lidar_Rs, lidar_ts, positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

In [ ]:
# Load satellite positions
sv_path = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'svs')
#svfile = os.path.join(sv_path, f'{kitti_seq}_saved_sats.csv')
sv_positions = load_sv_positions(gtpath, sv_path, kitti_seq, start_idx=start_idx)

In [ ]:
# For all traces (0018, 0027, 0028, 0034), the number of satellites in the first 2000 frames is constant
N_SATS = len(sv_positions[0])

In [ ]:
# Compute DOP
los = sv_positions[0]
los = los / np.linalg.norm(los, axis=1)[:,None]

In [ ]:
A = np.hstack((los, np.ones((N_SATS, 1))))
Q = np.linalg.inv(A.T @ A)
PDOP = np.sqrt(Q[0,0] + Q[1,1] + Q[2,2])
Q

In [ ]:
6 * np.diag(Q)[:3]

### Parameters

In [ ]:
TRAJLEN = 2000

MAX_BIAS = 50  # [m]
ATTACK_START = TRAJLEN // 2

# Sliding window of size 100 which shifts by 10 each iteration
N_SHIFT = 10
N_WINDOW = 100

GPS_RATE = 10  # ratio of odometry to GPS measurements

PR_SIGMA = 6

# Lidar odometry covariance
ODOM_R_SIGMA = 0.01  # 
ODOM_T_SIGMA = 0.05  # [m]
ODOM_SIGMA = np.ones(6)
ODOM_SIGMA[:3] *= ODOM_R_SIGMA
ODOM_SIGMA[3:] *= ODOM_T_SIGMA

### Problem Setup 

In [ ]:
# Lidar odometry
lidar_odom = [None] * TRAJLEN
for i in range(TRAJLEN - 1):
    lidar_odom[i] = (lidar_Rs[i], lidar_ts[i])

### Spoofing

In [ ]:
from scipy.stats import chi2

# Compute threshold
alpha = 0.001  # False alarm (FA) rate
T = chi2.ppf(1-alpha, df=N_SATS*(N_WINDOW/GPS_RATE))
print("Threshold = ", T)

In [ ]:
1 - (1-alpha)**200

In [ ]:
chi2.ppf(1-alpha, df=(N_SATS-3)*(N_WINDOW/GPS_RATE))

In [ ]:
# Spoofed trajectory
spoofed_pos = gt_enu[:TRAJLEN].copy()

gps_spoofing_biases = np.zeros(TRAJLEN)  
gps_spoofing_biases[ATTACK_START:] = np.linspace(0, MAX_BIAS, TRAJLEN-ATTACK_START)  # Ramping attack

spoofed_pos[:,0] -= gps_spoofing_biases

# Spoofed ranges
PR_SIGMA = 6
spoofed_ranges = np.zeros((TRAJLEN, N_SATS))
for i in range(TRAJLEN):
    svs = sv_positions[i]
    for j in range(N_SATS):
        spoofed_ranges[i,j] = np.linalg.norm(spoofed_pos[i] - svs[j]) + np.random.normal(0, PR_SIGMA)

In [ ]:
from chimera_fgo.symforce.factor_graph import fgo_eph

PR_SIGMA = 6

graph_positions = np.zeros((TRAJLEN, 3))
# graph_rotations = np.zeros((TRAJLEN, 3))
graph_attitudes = np.zeros((TRAJLEN, 3))
init_poses = [sf.Pose3.identity()] * N_WINDOW
init_poses[0] = sf.Pose3(sf.Rot3.from_rotation_matrix(gt_Rs[0]), sf.V3(gt_enu[0]))
e_ranges = np.zeros((N_WINDOW//GPS_RATE, N_SATS))
qs = np.zeros(TRAJLEN//GPS_RATE)

for k in (pbar := tqdm(range((TRAJLEN - N_WINDOW) // N_SHIFT))):

    window = slice(N_SHIFT * k, N_SHIFT * k + N_WINDOW)
    odom = lidar_odom[window]
    ranges = spoofed_ranges[window]
    #odom_sigmas = lidar_sigmas[window]

    satpos_enu = np.array(sv_positions[window])
    satpos = [[sf.V3(satpos_enu[i][j]) for j in range(N_SATS)] for i in range(N_WINDOW)]

    result = fgo_eph(init_poses, satpos, ranges, odom, PR_SIGMA, ODOM_SIGMA, GPS_RATE, fix_first_pose=True, debug=False)

    # Extract optimized positions
    window_positions = np.zeros((N_WINDOW, 3))
    window_attitudes = np.zeros((N_WINDOW, 3))
    for i in range(N_WINDOW):
        pose = result.optimized_values["poses"][i]
        window_positions[i] = pose.position().flatten()   
        window_attitudes[i] = pose.rotation().to_yaw_pitch_roll().flatten()   

    # Save trajectory
    graph_positions[N_SHIFT*k:N_SHIFT*(k+1)] = window_positions[:N_SHIFT]
    graph_attitudes[N_SHIFT*k:N_SHIFT*(k+1)] = window_attitudes[:N_SHIFT]

    # Update initial poses
    init_poses[:-N_SHIFT] = result.optimized_values["poses"][N_SHIFT:]

    # Use lidar odometry to initialize latest new poses
    n_pose = result.optimized_values["poses"][-1]
    n_pose = sf.Pose3(sf.Rot3.from_rotation_matrix(n_pose.R.to_rotation_matrix()), sf.V3(n_pose.t))
    for i in range(N_SHIFT):
        lidar_T = sf.Pose3(sf.Rot3.from_rotation_matrix(lidar_Rs[N_SHIFT*k + N_WINDOW + i]), sf.V3(lidar_ts[N_SHIFT*k + N_WINDOW + i]))
        n_pose = lidar_T * n_pose
        init_poses[-N_SHIFT+i] = n_pose

    # Compute test statistic
    for i in range(N_WINDOW//GPS_RATE):
        svs = sv_positions[N_SHIFT * k + i * GPS_RATE]
        for j in range(N_SATS):
            e_ranges[i,j] = np.linalg.norm(window_positions[::GPS_RATE][i] - svs[j])
    q = np.sum((ranges[::GPS_RATE] - e_ranges)**2) / PR_SIGMA**2
    pbar.set_description(f"q = {q:.2f}")
    qs[k] = q

    # Mitigation
    if q > T:
        print("Attack detected at ", N_SHIFT * k)
        PR_SIGMA = 1e5

# # End of trajectory, save all the positions from the current window
# for i in range(N_WINDOW):
#     idx = TRAJLEN - N_WINDOW + i
#     graph_positions[idx] = result.optimized_values["poses"][i].position().flatten()
OPT_TRAJLEN = N_SHIFT*(k+1)

In [ ]:
OPT_TRAJLEN = N_SHIFT*(k+1)
graph_positions_plot = graph_positions[:OPT_TRAJLEN]
gt_enu_plot = gt_enu[:OPT_TRAJLEN]
qs_plot = qs[:OPT_TRAJLEN//N_SHIFT]

In [ ]:
pose.rotation().to_yaw_pitch_roll().flatten()

In [ ]:
pose.rotation().to_yaw_pitch_roll().flatten()[[2,1,0]]

In [ ]:
graph_attitudes

In [ ]:
from chimera_fgo.util.geometry import R_to_euler

R_to_euler(gt_Rs[0])

In [ ]:
gt_attitudes[0]

In [ ]:
init_R = sf.Rot3.from_rotation_matrix(gt_Rs[0])
type(init_R.to_yaw_pitch_roll()[0])

In [ ]:
init_R.to_yaw_pitch_roll()

In [ ]:
np.arctan2(0.861908124186026, 0.50644267722056)

In [ ]:
-np.arcsin(0.0251037875986254)

In [ ]:
np.arctan2(0.0405591713410341, 0.998861728903622)

In [ ]:
import plotly.figure_factory as ff

N = 100
d = 2

#thetas = gt_attitudes[:,2]
thetas = graph_attitudes[:,2]

fig = ff.create_quiver(x=gt_enu[0:N:d,0], y=gt_enu[0:N:d,1], u=np.cos(thetas[0:N:d]), v=np.sin(thetas[0:N:d]), scale=5.0)
gt_traj = go.Scatter(x=gt_enu[0:N:d,0], y=gt_enu[0:N:d,1], hovertext=np.arange(N), name='Ground-truth')
fig.add_trace(gt_traj)
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Plot test statistic and threshold
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(qs_plot)), y=qs_plot, name='Test statistic'))
fig.add_trace(go.Scatter(x=np.arange(len(qs_plot)), y=T*np.ones(len(qs_plot)), name='Threshold', line=dict(color='red', dash='dash')))
# Add vertical line at start of spoofing attack
fig.add_shape(type="line", x0=ATTACK_START/GPS_RATE, y0=-20, x1=ATTACK_START/GPS_RATE, y1=200, line=dict(color="black", width=2, dash="dash"))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='Test statistic')
#fig.update_layout(legend=dict(x=0.75, y=0.98), font=dict(size=18))
fig.show()

In [ ]:
fgo_traj = go.Scatter(x=graph_positions_plot[:,0], y=graph_positions_plot[:,1], hovertext=np.arange(TRAJLEN), name='FGO trajectory')
gt_traj = go.Scatter(x=gt_enu_plot[:,0], y=gt_enu_plot[:,1], hovertext=np.arange(OPT_TRAJLEN), name='Ground-truth')
spoof_traj = go.Scatter(x=spoofed_pos[:OPT_TRAJLEN,0], y=spoofed_pos[:OPT_TRAJLEN,1], hovertext=np.arange(OPT_TRAJLEN), name='Spoofed')
start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='blue'), showlegend=False)
fig = go.Figure(data=[gt_traj, fgo_traj, spoof_traj, start])
fig.update_layout(width=700, height=700, xaxis_title='East [m]', yaxis_title='North [m]')
# Move legend into plot
fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)
fig.show()

In [ ]:
# Plot x and y errors over time
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=graph_positions_plot[:,0] - gt_enu_plot[:,0], name='x error'))
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=graph_positions_plot[:,1] - gt_enu_plot[:,1], name='y error'))
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=graph_positions_plot[:,2] - gt_enu_plot[:,2], name='z error'))
# Plot RMSE
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=np.sqrt(np.mean((graph_positions_plot - gt_enu_plot)**2, axis=1)), name='RMSE'))
#fig.add_shape(type="line", x0=ATTACK_START, y0=-20, x1=ATTACK_START, y1=90, line=dict(color="black", width=2, dash="dash"))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='Error [m]')
fig.update_layout(font=dict(size=15))
fig.show()


In [ ]:
from plotly.subplots import make_subplots

# Plot x, y, z errors in subplots on top of each other
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05)
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=graph_positions_plot[:,0] - gt_enu_plot[:,0], name='x error'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=graph_positions_plot[:,1] - gt_enu_plot[:,1], name='y error'), row=2, col=1)
fig.add_trace(go.Scatter(x=np.arange(OPT_TRAJLEN), y=graph_positions_plot[:,2] - gt_enu_plot[:,2], name='z error'), row=3, col=1)
fig.update_layout(width=1200, height=700)
fig.update_layout(font=dict(size=15))
fig.show()

In [ ]:
# RMSE error
rmse_xyz = np.sqrt(np.mean((graph_positions_plot - gt_enu_plot)**2, axis=0))
print("RMSE (xyz): ", rmse_xyz)
print("RMSE (overall): ", rmse_xyz.mean())

In [ ]:
# Mean error
mean_error = np.mean((graph_positions_plot - gt_enu_plot), axis=0)
std_error = np.std((graph_positions_plot - gt_enu_plot), axis=0)
print("Mean error (xyz): ", mean_error)
print("Std error (xyz): ", std_error)

In [ ]:
# Rotational error
gt_attitudes_plot = gt_attitudes[:OPT_TRAJLEN]
